In [1]:
import os
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.cross_validation import train_test_split

import tensorflow as tf

/home/abhishek/Desktop/Projects/tf/venv/local/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
os.chdir('/home/abhishek/Desktop/Projects/tf/yet_another_ML_tutorial/coding_exercise/')
raw_data = pd.read_csv("./CreditDataset.csv", header=None)
print "Shape of original data frame:", raw_data.shape

Shape of original data frame: (1000, 21)


In [3]:
# Get data types
print raw_data.dtypes
obj_df = raw_data.select_dtypes(include=['object']).copy()
print "Shape of object data frame:", obj_df.shape
int_df = raw_data.select_dtypes(include=['int64']).copy()
print "Shape of int64 data frame:", int_df.shape
print "Type of int data frame:", type(int_df)

# Check for null values in the columns containing categorical variables
print obj_df[obj_df.isnull().any(axis=1)]

0     object
1      int64
2     object
3     object
4      int64
5     object
6     object
7      int64
8     object
9     object
10     int64
11    object
12     int64
13    object
14    object
15     int64
16    object
17     int64
18    object
19    object
20     int64
dtype: object
Shape of object data frame: (1000, 13)
Shape of int64 data frame: (1000, 8)
Type of int data frame: <class 'pandas.core.frame.DataFrame'>
Empty DataFrame
Columns: [0, 2, 3, 5, 6, 8, 9, 11, 13, 14, 16, 18, 19]
Index: []


In [4]:
# One hot encoding of the columns containing categorical variables
# Label encoder
# 1. INSTANTIATE
# encode labels with value between 0 and n_classes-1.
le = preprocessing.LabelEncoder()
# FIT AND TRANSFORM. use df.apply() to apply le.fit_transform to all columns
le_obj_df = obj_df.apply(le.fit_transform)
# print raw_data.select_dtypes(include=['object']).head(5)
# print le_obj_df.head()

# One hot encoding of categorical variables
# 1. INSTANTIATE
encode_object = preprocessing.OneHotEncoder()
# 2. FIT
encode_object.fit(le_obj_df)
# 3. Transform
onehotlabels = encode_object.transform(le_obj_df).toarray()
print onehotlabels.shape
print type(onehotlabels)

# Merge the int64 data frame with the one hot labels
np_int_df = int_df.as_matrix()
print np_int_df.shape
processed_data = np.concatenate([onehotlabels, np_int_df], axis=1)
print processed_data.shape

# print processed_data[:,-1]
print processed_data.dtype


# Encode output for NN
# encode_output = preprocessing.OneHotEncoder()
# encode_output.fit(processed_data[:,-1])
# output_onehotlabels = encode_output.transform(processed_data[:,-1])
# print "Shape of encoded one hot labels:", output_onehotlabels.shape
#

raw_labels = np.array(processed_data[:,-1]).astype(int)
encoded_labels = np.zeros((processed_data[:,-1].shape[0], 2))
encoded_labels[np.arange(processed_data[:,-1].shape[0]), raw_labels-1] = 1


processed_data = processed_data[:,0:61]
processed_data = np.concatenate([processed_data, encoded_labels], axis=1)
print processed_data.shape


(1000, 54)
<type 'numpy.ndarray'>
(1000, 8)
(1000, 62)
float64
(1000, 63)


In [5]:
# Get test train split
X_train, X_test, y_train, y_test = train_test_split(processed_data[:, 0:61],
                                                    processed_data[:, 61:63],
                                                    test_size=0.3,
                                                    random_state=42)


In [6]:
print X_train.shape, X_test.shape

(700, 61) (300, 61)


In [7]:
print y_train.shape, y_test.shape

(700, 2) (300, 2)


In [8]:
def random_batch(dataset, batch_size):
    sample = dataset[np.random.choice(dataset.shape[0], batch_size, replace=False),:]
    last_col_index = dataset.shape[1]-2
    x = sample[:,0:last_col_index]
    y = sample[:,last_col_index:last_col_index+2]
    return (x, y)

In [9]:
# Parameters
learning_rate = 0.001
num_steps = 500000
batch_size = 100
display_step = 100

# Network Parameters
n_hidden_1 = 256 # 1st layer number of neurons
n_hidden_2 = 256 # 2nd layer number of neurons
num_input = X_train.shape[1] 
num_classes = y_train.shape[1]

print "Number of input:", num_input
print "Number of classes:", num_classes

# tf Graph input
X = tf.placeholder("float", [None, num_input])
Y = tf.placeholder("float", [None, num_classes])

Number of input: 61
Number of classes: 2


In [10]:
# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([num_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, num_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([num_classes]))
}

In [11]:
# Create model
def neural_net(x):
    # Hidden fully connected layer with 256 neurons
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    # Hidden fully connected layer with 256 neurons
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    # Output fully connected layer with a neuron for each class
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

In [12]:
# Construct model
logits = neural_net(X)

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# Evaluate model (with test logits, for dropout to be disabled)
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

In [13]:
# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    for step in range(1, num_steps+1):
        batch_x, batch_y = random_batch(np.concatenate([X_train, y_train], axis=1), 
                                        batch_size) 
        # Run optimization op (backprop)
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x,
                                                                 Y: batch_y})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))

    print("Optimization Finished!")

    # Calculate accuracy for MNIST test images
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={X: X_test,
                                      Y: y_test}))

Step 1, Minibatch Loss= 780731.0625, Training Accuracy= 0.210
Step 100, Minibatch Loss= 12819.0850, Training Accuracy= 0.700
Step 200, Minibatch Loss= 9483.3389, Training Accuracy= 0.650
Step 300, Minibatch Loss= 29499.9434, Training Accuracy= 0.300
Step 400, Minibatch Loss= 4385.1675, Training Accuracy= 0.480
Step 500, Minibatch Loss= 5533.4785, Training Accuracy= 0.470
Step 600, Minibatch Loss= 35744.6406, Training Accuracy= 0.290
Step 700, Minibatch Loss= 21940.8945, Training Accuracy= 0.350
Step 800, Minibatch Loss= 3054.1096, Training Accuracy= 0.710
Step 900, Minibatch Loss= 8765.0000, Training Accuracy= 0.770
Step 1000, Minibatch Loss= 11961.2773, Training Accuracy= 0.460
Step 1100, Minibatch Loss= 11436.5273, Training Accuracy= 0.710
Step 1200, Minibatch Loss= 16345.3037, Training Accuracy= 0.670
Step 1300, Minibatch Loss= 7451.9771, Training Accuracy= 0.720
Step 1400, Minibatch Loss= 8149.7036, Training Accuracy= 0.680
Step 1500, Minibatch Loss= 5334.7031, Training Accuracy= 0

Step 13000, Minibatch Loss= 2619.3875, Training Accuracy= 0.710
Step 13100, Minibatch Loss= 2483.3774, Training Accuracy= 0.710
Step 13200, Minibatch Loss= 4710.4956, Training Accuracy= 0.770
Step 13300, Minibatch Loss= 8376.9590, Training Accuracy= 0.540
Step 13400, Minibatch Loss= 782.0744, Training Accuracy= 0.900
Step 13500, Minibatch Loss= 3133.8503, Training Accuracy= 0.780
Step 13600, Minibatch Loss= 3582.4111, Training Accuracy= 0.700
Step 13700, Minibatch Loss= 1626.2415, Training Accuracy= 0.770
Step 13800, Minibatch Loss= 2309.3552, Training Accuracy= 0.800
Step 13900, Minibatch Loss= 1377.4655, Training Accuracy= 0.770
Step 14000, Minibatch Loss= 46863.2812, Training Accuracy= 0.320
Step 14100, Minibatch Loss= 1754.2423, Training Accuracy= 0.710
Step 14200, Minibatch Loss= 4439.7363, Training Accuracy= 0.650
Step 14300, Minibatch Loss= 4306.2949, Training Accuracy= 0.720
Step 14400, Minibatch Loss= 7048.6660, Training Accuracy= 0.580
Step 14500, Minibatch Loss= 4788.7246, T

Step 25800, Minibatch Loss= 1755.3243, Training Accuracy= 0.720
Step 25900, Minibatch Loss= 1564.6893, Training Accuracy= 0.720
Step 26000, Minibatch Loss= 2916.7175, Training Accuracy= 0.690
Step 26100, Minibatch Loss= 4729.8745, Training Accuracy= 0.760
Step 26200, Minibatch Loss= 863.5269, Training Accuracy= 0.840
Step 26300, Minibatch Loss= 1383.2690, Training Accuracy= 0.810
Step 26400, Minibatch Loss= 6285.6689, Training Accuracy= 0.700
Step 26500, Minibatch Loss= 4307.7651, Training Accuracy= 0.670
Step 26600, Minibatch Loss= 4202.7676, Training Accuracy= 0.670
Step 26700, Minibatch Loss= 2027.2784, Training Accuracy= 0.700
Step 26800, Minibatch Loss= 1388.1200, Training Accuracy= 0.790
Step 26900, Minibatch Loss= 8741.0010, Training Accuracy= 0.530
Step 27000, Minibatch Loss= 3790.5457, Training Accuracy= 0.620
Step 27100, Minibatch Loss= 7353.0845, Training Accuracy= 0.560
Step 27200, Minibatch Loss= 1695.6428, Training Accuracy= 0.770
Step 27300, Minibatch Loss= 5342.2983, Tr

Step 38600, Minibatch Loss= 5026.7100, Training Accuracy= 0.700
Step 38700, Minibatch Loss= 4404.1914, Training Accuracy= 0.730
Step 38800, Minibatch Loss= 3941.0947, Training Accuracy= 0.710
Step 38900, Minibatch Loss= 1771.4971, Training Accuracy= 0.730
Step 39000, Minibatch Loss= 11896.6680, Training Accuracy= 0.480
Step 39100, Minibatch Loss= 1302.8524, Training Accuracy= 0.780
Step 39200, Minibatch Loss= 4065.0706, Training Accuracy= 0.620
Step 39300, Minibatch Loss= 1490.2064, Training Accuracy= 0.700
Step 39400, Minibatch Loss= 3127.0461, Training Accuracy= 0.610
Step 39500, Minibatch Loss= 5597.4888, Training Accuracy= 0.580
Step 39600, Minibatch Loss= 4656.1445, Training Accuracy= 0.580
Step 39700, Minibatch Loss= 5073.2349, Training Accuracy= 0.740
Step 39800, Minibatch Loss= 3785.4355, Training Accuracy= 0.650
Step 39900, Minibatch Loss= 3540.4272, Training Accuracy= 0.780
Step 40000, Minibatch Loss= 8083.3813, Training Accuracy= 0.780
Step 40100, Minibatch Loss= 2012.8964, 

Step 51400, Minibatch Loss= 16327.9873, Training Accuracy= 0.410
Step 51500, Minibatch Loss= 2739.8730, Training Accuracy= 0.740
Step 51600, Minibatch Loss= 1569.3744, Training Accuracy= 0.820
Step 51700, Minibatch Loss= 1066.8926, Training Accuracy= 0.760
Step 51800, Minibatch Loss= 1112.8453, Training Accuracy= 0.830
Step 51900, Minibatch Loss= 1040.6414, Training Accuracy= 0.790
Step 52000, Minibatch Loss= 2502.0645, Training Accuracy= 0.730
Step 52100, Minibatch Loss= 985.3452, Training Accuracy= 0.720
Step 52200, Minibatch Loss= 9413.3564, Training Accuracy= 0.700
Step 52300, Minibatch Loss= 1731.7366, Training Accuracy= 0.790
Step 52400, Minibatch Loss= 5318.5215, Training Accuracy= 0.510
Step 52500, Minibatch Loss= 7568.2524, Training Accuracy= 0.480
Step 52600, Minibatch Loss= 2399.4111, Training Accuracy= 0.730
Step 52700, Minibatch Loss= 7767.5444, Training Accuracy= 0.530
Step 52800, Minibatch Loss= 2259.7246, Training Accuracy= 0.780
Step 52900, Minibatch Loss= 4936.8398, T

...

Step 448200, Minibatch Loss= 326.6830, Training Accuracy= 0.730
Step 448300, Minibatch Loss= 860.2729, Training Accuracy= 0.710
Step 448400, Minibatch Loss= 365.6197, Training Accuracy= 0.770
Step 448500, Minibatch Loss= 162.6965, Training Accuracy= 0.740
Step 448600, Minibatch Loss= 76.9304, Training Accuracy= 0.740
Step 448700, Minibatch Loss= 312.9701, Training Accuracy= 0.730
Step 448800, Minibatch Loss= 156.4412, Training Accuracy= 0.770
Step 448900, Minibatch Loss= 197.9276, Training Accuracy= 0.780
Step 449000, Minibatch Loss= 413.0977, Training Accuracy= 0.620
Step 449100, Minibatch Loss= 307.0417, Training Accuracy= 0.720
Step 449200, Minibatch Loss= 232.2147, Training Accuracy= 0.630
Step 449300, Minibatch Loss= 644.9999, Training Accuracy= 0.560
Step 449400, Minibatch Loss= 111.6966, Training Accuracy= 0.840
Step 449500, Minibatch Loss= 153.1954, Training Accuracy= 0.770
Step 449600, Minibatch Loss= 367.5784, Training Accuracy= 0.740
Step 449700, Minibatch Loss= 215.6590, Tr

Step 461100, Minibatch Loss= 214.5684, Training Accuracy= 0.680
Step 461200, Minibatch Loss= 139.2149, Training Accuracy= 0.790
Step 461300, Minibatch Loss= 1020.3420, Training Accuracy= 0.630
Step 461400, Minibatch Loss= 189.9217, Training Accuracy= 0.690
Step 461500, Minibatch Loss= 450.7400, Training Accuracy= 0.630
Step 461600, Minibatch Loss= 135.5321, Training Accuracy= 0.760
Step 461700, Minibatch Loss= 116.7761, Training Accuracy= 0.720
Step 461800, Minibatch Loss= 406.1417, Training Accuracy= 0.730
Step 461900, Minibatch Loss= 141.6522, Training Accuracy= 0.780
Step 462000, Minibatch Loss= 453.0263, Training Accuracy= 0.740
Step 462100, Minibatch Loss= 144.5575, Training Accuracy= 0.790
Step 462200, Minibatch Loss= 568.0348, Training Accuracy= 0.720
Step 462300, Minibatch Loss= 98.7857, Training Accuracy= 0.800
Step 462400, Minibatch Loss= 114.7432, Training Accuracy= 0.800
Step 462500, Minibatch Loss= 1009.6775, Training Accuracy= 0.540
Step 462600, Minibatch Loss= 210.3559, 

Step 474000, Minibatch Loss= 221.1462, Training Accuracy= 0.720
Step 474100, Minibatch Loss= 146.7843, Training Accuracy= 0.720
Step 474200, Minibatch Loss= 642.6679, Training Accuracy= 0.630
Step 474300, Minibatch Loss= 254.3006, Training Accuracy= 0.710
Step 474400, Minibatch Loss= 548.8092, Training Accuracy= 0.560
Step 474500, Minibatch Loss= 136.9474, Training Accuracy= 0.760
Step 474600, Minibatch Loss= 642.5758, Training Accuracy= 0.700
Step 474700, Minibatch Loss= 127.7966, Training Accuracy= 0.770
Step 474800, Minibatch Loss= 148.5255, Training Accuracy= 0.730
Step 474900, Minibatch Loss= 738.4686, Training Accuracy= 0.620
Step 475000, Minibatch Loss= 331.2645, Training Accuracy= 0.820
Step 475100, Minibatch Loss= 196.8170, Training Accuracy= 0.620
Step 475200, Minibatch Loss= 336.3759, Training Accuracy= 0.630
Step 475300, Minibatch Loss= 847.3517, Training Accuracy= 0.440
Step 475400, Minibatch Loss= 114.9837, Training Accuracy= 0.810
Step 475500, Minibatch Loss= 529.1583, T

Step 486900, Minibatch Loss= 237.9860, Training Accuracy= 0.730
Step 487000, Minibatch Loss= 178.6477, Training Accuracy= 0.720
Step 487100, Minibatch Loss= 75.1300, Training Accuracy= 0.760
Step 487200, Minibatch Loss= 143.3118, Training Accuracy= 0.790
Step 487300, Minibatch Loss= 220.5506, Training Accuracy= 0.830
Step 487400, Minibatch Loss= 349.1689, Training Accuracy= 0.660
Step 487500, Minibatch Loss= 103.1955, Training Accuracy= 0.830
Step 487600, Minibatch Loss= 84.5978, Training Accuracy= 0.780
Step 487700, Minibatch Loss= 235.0760, Training Accuracy= 0.760
Step 487800, Minibatch Loss= 148.9738, Training Accuracy= 0.790
Step 487900, Minibatch Loss= 142.6813, Training Accuracy= 0.690
Step 488000, Minibatch Loss= 526.4341, Training Accuracy= 0.710
Step 488100, Minibatch Loss= 157.2884, Training Accuracy= 0.680
Step 488200, Minibatch Loss= 73.0039, Training Accuracy= 0.810
Step 488300, Minibatch Loss= 1411.3501, Training Accuracy= 0.690
Step 488400, Minibatch Loss= 336.3120, Tra

Step 499800, Minibatch Loss= 101.3069, Training Accuracy= 0.800
Step 499900, Minibatch Loss= 38.0037, Training Accuracy= 0.850
Step 500000, Minibatch Loss= 94.8972, Training Accuracy= 0.740
Optimization Finished!
('Testing Accuracy:', 0.77666664)
